# Writing Instances: Upsert

We assume that you have [generated a SDK](generation.html) for the `Windmill` model and have a client ready to go.

The SDK generated by `SDK` supports nested upsert

In [1]:
import warnings
warnings.filterwarnings('ignore')
# This is just to enable improting the generated SDK from the examples folder in the pygen repository
import sys
from tests.constants import REPO_ROOT
sys.path.append(str(REPO_ROOT / "examples" ))

In [2]:
from windmill import WindmillClient

In [3]:
client = WindmillClient.from_toml("config.toml")

## Constructing new Windmill

When constructing a new Windmill we need to use the generated data classes from `pygen`. 
We can import this as follows:

In [4]:
from cognite.client.data_classes import TimeSeries
from windmill.data_classes import WindmillWrite, BladeWrite, NacelleWrite, RotorWrite

The location of the data classes is determined by the parameter `top_level_package` which is set when you generate the SDK,
either using the `generate_sdk_notebook` (simplified wrapper around) or `generate_sdk`. If you don't set it, it will be 
default set to the external_id of the data model converted to snake_case. For this example, the `external_id=Windmill`
thus the `top_level_package = windmill`.

Lets construct a new windmill with TimeSeries. Note the example below is not complete (some TimeSeries and components are missing), 
but is kept short to make it easier to grasp

In [5]:
new_windmill = WindmillWrite(
    external_id="windmill:demo",
    capacity=10.0,
    windfarm="Fornebu",
    name="Windmill ATH",
    rotor=RotorWrite(
        external_id="windmill:demo:rotor",
        rotor_speed_controller=TimeSeries(external_id="windmill:demo:rotor:speed_controller"),
        rpm_low_speed_shaft=TimeSeries(external_id="windmill:demo:rotor:rpm_low_speed_shaft"),
    ),
    nacelle=NacelleWrite(
        external_id="windmill:demo:nacelle",
        acc_from_back_side_x=TimeSeries(external_id="windmill:demo:acc_from_back_side_x"),
        acc_from_back_side_y=TimeSeries(external_id="windmill:demo:acc_from_back_side_y"),
        acc_from_back_side_z=TimeSeries(external_id="windmill:demo:acc_from_back_side_z"),
    ),
    blades=[
        BladeWrite(
            external_id="windmill:demo:blade1",
            is_damaged=False,
            name="Blade 1",
        ),
        BladeWrite(
            external_id="windmill:demo:blade2",
            is_damaged=False,
            name="Blade 2",
        ),
        BladeWrite(
            external_id="windmill:demo:blade3",
            is_damaged=True,
            name="Blade 3",
        ),
    ]
)

When writing nested data we can specify edges either with an external id for the end node, or another data class.

The advangage of using a nested data class is that we can express edges without being explicit. In the example above, we are able to express that the blades `Blade 1-3` are connected to the windmill `windmill ATH` and that the `windmill ATH` is also linked to a nacelle and rotor.

## Inspecting Resources to create

We can inspect the nodes, edges and other resources that will be created by using the `.to_instances_write` on the new windmill object.

In [6]:
resources = new_windmill.to_instances_write()

In [7]:
len(resources.nodes), len(resources.edges), len(resources.time_series)

(6, 3, 5)

In [8]:
resources.nodes

,space,instance_type,external_id,sources
0,windmill-instances,node,windmill:demo,"[{'properties': {'capacity': 10.0, 'nacelle': ..."
1,windmill-instances,node,windmill:demo:blade1,"[{'properties': {'is_damaged': False, 'name': ..."
2,windmill-instances,node,windmill:demo:blade2,"[{'properties': {'is_damaged': False, 'name': ..."
3,windmill-instances,node,windmill:demo:blade3,"[{'properties': {'is_damaged': True, 'name': '..."
4,windmill-instances,node,windmill:demo:nacelle,[{'properties': {'acc_from_back_side_x': 'wind...
5,windmill-instances,node,windmill:demo:rotor,[{'properties': {'rotor_speed_controller': 'wi...


In [9]:
resources.edges

,space,instance_type,external_id,type,start_node,end_node
0,windmill-instances,edge,windmill:demo:windmill:demo:blade1,"{'space': 'power-models', 'external_id': 'Wind...","{'space': 'windmill-instances', 'external_id':...","{'space': 'windmill-instances', 'external_id':..."
1,windmill-instances,edge,windmill:demo:windmill:demo:blade2,"{'space': 'power-models', 'external_id': 'Wind...","{'space': 'windmill-instances', 'external_id':...","{'space': 'windmill-instances', 'external_id':..."
2,windmill-instances,edge,windmill:demo:windmill:demo:blade3,"{'space': 'power-models', 'external_id': 'Wind...","{'space': 'windmill-instances', 'external_id':...","{'space': 'windmill-instances', 'external_id':..."


In [10]:
resources.time_series

,external_id
0,windmill:demo:acc_from_back_side_x
1,windmill:demo:acc_from_back_side_y
2,windmill:demo:acc_from_back_side_z
3,windmill:demo:rotor:speed_controller
4,windmill:demo:rotor:rpm_low_speed_shaft


## Creating new Windmill

**Optinal Reading**: Why `client.upsert` and not `client.windmill.upsert`?

In contrast from other methods, the `.upsert` method is on the `client` instead of the individual API class. So instead of `client.windmill.upsert`, we use `client.upsert`. 

The reason for this is that the `new_windmill` we created above is enhanced by `pygen` with all the information needed to write it correctly to our data model. This means that all `.upsert` methods are the same, this is in contrast to methods such as `.list` and `.retrieve` which are specialized for each data type.

Furthermore, the reason for not duplicating the `.upsert` methods on each API class (`client.windmill.upsert`, `client.blade.upsert`, and so on) is that encourages an anti-pattern (bad practice), in which nodes and edges are created in small batches. It is much more efficient to create all nodes and edges in as few batches as possible.

In [11]:
created = client.upsert(new_windmill)

Note that the call above created 6 nodes, 3 edges and 5 time series:

In [12]:
created.nodes

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,windmill-instances,node,windmill:demo,1,True,2024-02-10 09:24:48.149,2024-02-10 09:24:48.149
1,windmill-instances,node,windmill:demo:blade1,1,True,2024-02-10 09:24:48.149,2024-02-10 09:24:48.149
2,windmill-instances,node,windmill:demo:blade2,1,True,2024-02-10 09:24:48.149,2024-02-10 09:24:48.149
3,windmill-instances,node,windmill:demo:blade3,1,True,2024-02-10 09:24:48.149,2024-02-10 09:24:48.149
4,windmill-instances,node,windmill:demo:nacelle,2,True,2024-02-10 09:24:48.149,2024-02-10 09:24:48.149
5,windmill-instances,node,windmill:demo:rotor,2,True,2024-02-10 09:24:48.149,2024-02-10 09:24:48.149


In [13]:
created.edges

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,windmill-instances,edge,windmill:demo:windmill:demo:blade1,1,True,2024-02-10 09:24:48.149,2024-02-10 09:24:48.149
1,windmill-instances,edge,windmill:demo:windmill:demo:blade2,1,True,2024-02-10 09:24:48.149,2024-02-10 09:24:48.149
2,windmill-instances,edge,windmill:demo:windmill:demo:blade3,1,True,2024-02-10 09:24:48.149,2024-02-10 09:24:48.149


And we can now see that the movie is now listed among the others.

In [14]:
created.time_series

,external_id,is_string,metadata,is_step,security_categories,id,created_time,last_updated_time
0,windmill:demo:acc_from_back_side_x,False,{},False,[],2621333712342203,2024-02-10 09:24:48.498,2024-02-10 09:24:48.498
1,windmill:demo:acc_from_back_side_y,False,{},False,[],3464215227079536,2024-02-10 09:24:48.498,2024-02-10 09:24:48.498
2,windmill:demo:acc_from_back_side_z,False,{},False,[],1758324661664066,2024-02-10 09:24:48.498,2024-02-10 09:24:48.498
3,windmill:demo:rotor:speed_controller,False,{},False,[],2457085969431205,2024-02-10 09:24:48.498,2024-02-10 09:24:48.498
4,windmill:demo:rotor:rpm_low_speed_shaft,False,{},False,[],4813008531011590,2024-02-10 09:24:48.498,2024-02-10 09:24:48.498


We can inspect the newly created windmill by calling retrieve with the external id

In [15]:
client.windmill.retrieve(new_windmill.external_id)

,value
space,windmill-instances
external_id,windmill:demo
data_record,"{'version': 1, 'last_updated_time': 2024-02-10..."
node_type,None
blades,"[windmill:demo:blade1, windmill:demo:blade2, w..."
capacity,10.0
metmast,None
nacelle,windmill:demo:nacelle
name,Windmill ATH
rotor,windmill:demo:rotor


## Creating from <code>JSON</code> Format

See the quick start guide [data population](../quickstart/ingestion.html) for an example of creating instances from `JSON`. 

# Deleting Instances

You can delete by passing and external ID or a sequence of external id to the delete method.

We can delete the newly created windmill 

In [16]:
client.windmill.list()

,external_id,node_type,blades,capacity,metmast,nacelle,name,rotor,windfarm
0,hornsea_1_mill_3,None,"[blade:1, blade:2, blade:3]",7.0,None,nacelle:1,hornsea_1_mill_3,rotor:1,Hornsea 1
1,hornsea_1_mill_2,None,"[blade:4, blade:5, blade:6]",7.0,None,nacelle:2,hornsea_1_mill_2,rotor:2,Hornsea 1
2,hornsea_1_mill_1,None,"[blade:7, blade:8, blade:9]",7.0,None,nacelle:3,hornsea_1_mill_1,rotor:3,Hornsea 1
3,hornsea_1_mill_4,None,"[blade:10, blade:11, blade:12]",7.0,None,nacelle:4,hornsea_1_mill_4,rotor:4,Hornsea 1
4,hornsea_1_mill_5,None,"[blade:13, blade:14, blade:15]",7.0,None,nacelle:5,hornsea_1_mill_5,rotor:5,Hornsea 1
5,windmill:demo,None,"[windmill:demo:blade1, windmill:demo:blade2, w...",10.0,None,windmill:demo:nacelle,Windmill ATH,windmill:demo:rotor,Fornebu


Same as `.upsert`, the delete method is located on the client and not the API class.

In [17]:
client.delete("windmill:demo")

InstancesDeleteResult(nodes=[NodeId(space='windmill-instances', external_id='windmill:demo')], edges=[])

After the delete call the new windmill is gone

In [18]:
client.windmill.list()

,external_id,node_type,blades,capacity,metmast,nacelle,name,rotor,windfarm
0,hornsea_1_mill_3,None,"[blade:1, blade:2, blade:3]",7.0,None,nacelle:1,hornsea_1_mill_3,rotor:1,Hornsea 1
1,hornsea_1_mill_2,None,"[blade:4, blade:5, blade:6]",7.0,None,nacelle:2,hornsea_1_mill_2,rotor:2,Hornsea 1
2,hornsea_1_mill_1,None,"[blade:7, blade:8, blade:9]",7.0,None,nacelle:3,hornsea_1_mill_1,rotor:3,Hornsea 1
3,hornsea_1_mill_4,None,"[blade:10, blade:11, blade:12]",7.0,None,nacelle:4,hornsea_1_mill_4,rotor:4,Hornsea 1
4,hornsea_1_mill_5,None,"[blade:13, blade:14, blade:15]",7.0,None,nacelle:5,hornsea_1_mill_5,rotor:5,Hornsea 1


In [19]:
# Cleanup windmill and timeseries
cdf = client.windmill._client

cdf.data_modeling.instances.delete(nodes=created.nodes.as_ids(), edges=created.edges.as_ids());

cdf.time_series.delete(external_id=created.time_series.as_external_ids(), ignore_unknown_ids=True);